In [53]:
import tensorflow as tf
import numpy as np

%matplotlib inline
import pylab

import matplotlib.pyplot as plt
import scipy
import scipy.io as sio
from sklearn.model_selection import train_test_split

image_by_pixel = sio.loadmat('combined_img.mat')
label_by_pixel = sio.loadmat('combined_lab.mat')

test_image_by_pixel = sio.loadmat('s3d2_jpgimage.mat')
test_label_by_pixel = sio.loadmat('s3d2_jpglabel.mat')


In [54]:
down_sample_rate=4
def image_process(image, label, down_sample_rate=4):
    images = image['allimages'][::down_sample_rate ,::down_sample_rate , :]  # downsample the image
    labels = label['disc'][::down_sample_rate ,::down_sample_rate , :]       # downsample the pixel
    image_length1 = images.shape[0]
    image_length2 = images.shape[1]
    num_image = images.shape[2]                # number of images
    image_size = image_length1*image_length2   # image size 
    
    label_whole = np.zeros((num_image, image_size))#the label of whether the image is all trash or not
    images_processed = np.zeros((num_image, image_size))    
    for _ in range(num_image):
        label_whole[_] = labels[:,:,_].reshape(image_size)
        images_processed[_] = images[:,:,_].reshape(image_size)
    return images_processed, label_whole, image_size, image_length1

train_image, train_label, image_size, image_length = image_process(image_by_pixel,label_by_pixel, down_sample_rate)

test_image, test_label, _, _ = image_process(test_image_by_pixel, test_label_by_pixel, down_sample_rate)

########important ###############
output_size = image_size


In [55]:
# define variable placeholder for tf
x = tf.placeholder('float',[None, image_size])
y = tf.placeholder('float', [None, output_size])

In [56]:
conv1_patch_len = 7
conv1_num_filters = 10

conv2_patch_len = 7
conv2_num_filters = 10

full_con_len = 1024

def conv2d(x,W):             # strides = 1
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding = 'SAME')

def maxpool2d(x):            #size of window    # movement of window
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
    #return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1])

def conv_neural_net(x):
    weights = {'W_conv1': tf.Variable(tf.random_normal([conv1_patch_len, conv1_patch_len, 1 ,conv1_num_filters], mean=0.0,stddev=0.02)),
               'W_conv2': tf.Variable(tf.random_normal([conv2_patch_len, conv2_patch_len, conv1_num_filters,conv2_num_filters ], mean=0.0,stddev=0.02)),
               #'W_fc' : tf.Variable(tf.random_normal([image_size//16*conv2_num_filters, full_con_len], mean=0.0,stddev=0.02)),
               'W_fc' : tf.Variable(tf.random_normal([image_size*conv2_num_filters, full_con_len], mean=0.0,stddev=0.02)),
               'out' : tf.Variable(tf.random_normal([full_con_len ,output_size], mean=0.0,stddev=0.02))}
    
    biases = {'B_conv1': tf.Variable(tf.random_normal([conv1_num_filters], mean=0.0,stddev=0.02)),
               'B_conv2': tf.Variable(tf.random_normal([conv2_num_filters ], mean=0.0,stddev=0.02)),
               'B_fc' : tf.Variable(tf.random_normal([full_con_len], mean=0.0,stddev=0.02)),
               'out' : tf.Variable(tf.random_normal([output_size], mean=0.0,stddev=0.02))}
    
    x = tf.reshape(x, shape = [-1, image_length,image_length, 1])
    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1'])+ biases['B_conv1'])
    #conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2'])+ biases['B_conv2'])
    #conv2 = maxpool2d(conv2)
    
    #fc = tf.reshape(conv2, [-1, image_size//16*conv2_num_filters ])
    fc = tf.reshape(conv2, [-1, image_size*conv2_num_filters ])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['B_fc'])
    
    output = tf.matmul(fc, weights['out']+biases['out'])
    # output = tf.nn.softmax(output)
    return output


In [74]:
def train_neural_network(x):
    prediction = conv_neural_net(x)
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=prediction, labels=y) )
    # double check what cost function to use
    # cost = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) 
    # cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    hm_epochs = 1 # number of epochs
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            _, epoch_loss = sess.run([optimizer, cost], feed_dict={x: train_image, y: train_label})
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)
            
            
        pred =tf.to_int32(tf.greater(prediction, 1), name='ToInt32')
        correct = tf.equal(pred , tf.to_int32(y, name='ToInt32'))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float')) 
        print(accuracy.eval({x:test_image, y:test_label}))
        # print(prediction.eval({x:train_image})[0][0:150], prediction.eval({x:train_image})[1][0:150])
        sio.savemat("prediction", {'pred': pred.eval({x:train_image})})

In [75]:
train_neural_network(x)

Epoch 1 completed out of 1 loss: 15.4761
0.884358


In [76]:
a= sio.loadmat('prediction.mat')
a

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu May  4 20:29:28 2017',
 '__version__': '1.0',
 'pred': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)}